In [1]:
#Import packages that are using
import pandas as pd
import numpy as np
import os.path
import os,sys
from datetime import datetime, timedelta
import pyodbc
import pymssql
import pypyodbc
import csv
import math
import logging
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
#Logging provide an extra set of eyes that are constantly looking at the flow and
#debug errors easily but also use the data to analyze the performance of the application 
#to plan for scaling or look at usage patterns to plan for marketing.
#Start Automation Process & save as Automation.log
logger = logging.getLogger("Automation")
logger.setLevel(logging.INFO)
if logger.hasHandlers():
    logger.handlers.clear()
file_handler = logging.FileHandler("Automation.log")
file_handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s:%(levelname)s:%(message)s')
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

logger.info("******** HeartBeat Automation Process *********")

In [3]:
#Assign seven days before today's date&time to today1
today1 = datetime.now() - timedelta(7)
#Assign six days before today's date&time to today1
tomorrow1 = datetime.now() - timedelta(6)

In [4]:
print(today1)

2022-05-04 14:21:18.323514


In [5]:
print(tomorrow1)

2022-05-05 14:21:18.323514


In [6]:
#Extract month/date/year from today1 to today
#Extract month/date/year from tomorrow1 to tomorrow
today = datetime.strftime(today1, '%m/%d/%Y')
tomorrow = datetime.strftime(tomorrow1, '%m/%d/%Y')

In [7]:
print(today)
print(tomorrow)

05/04/2022
05/05/2022


In [8]:
#Extract date/month/year from today1 to today_log
#Extract date/month/year from tomorrow1 to tomorrow_log
today_log = datetime.strftime(today1, '%d/%m/%Y')
tomorrow_log = datetime.strftime(tomorrow1, '%d/%m/%Y')

In [9]:
#Output info into Automation.log
logger.info("Pax prediction for date {} started.".format(today_log))

In [10]:
#Assign all data of Active_Flights.csv file for active_flight_data
active_flight_data = pd.read_csv(r'C:\Users\Lenovo\Active_Flights.csv')

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8524\3689624070.py:2: DtypeWarning: Columns (32,34) have mixed types. Specify dtype option on import or set low_memory=False.
  active_flight_data = pd.read_csv(r'C:\Users\Lenovo\Active_Flights.csv')


In [11]:
#Remove spaces before & after active_flight_data['standcode']
#Assign active_flight_data['BAY'] equal to active_flight_data['standcode']
active_flight_data['standcode'] = active_flight_data['standcode'].str.strip()
active_flight_data['BAY'] = active_flight_data['standcode']

In [12]:
#Assign active_flight equal to active_flight_data['modesequence'] that has 0 or 1, 
#active_flight_data['remarkcodeflight'] that has no value of 'FCL' &
#active_flight_data['iataservicetypecode'] has values of 'C','G','J']
active_flight = active_flight_data[(active_flight_data['modesequence'].isin([0,1])) & (~active_flight_data['remarkcodeflight'].isin(['FCL'])) & (active_flight_data['iataservicetypecode'].isin(['C','G','J']))]

In [13]:
#Assign selected_columns_1 equal to columns of active_flight[['afskey','flightno','tailno','aircrafttypecode','leg','station1','station2','station3','station4','station5','station6','iataservicetypecode','stad','atad','standcode','BAY','categorycode','remarkcodeflight']]
#Copy all data of selected_columns_1 into today_active_flight
selected_columns_1 = active_flight[['afskey','flightno','tailno','aircrafttypecode','leg','station1','station2','station3','station4','station5','station6','iataservicetypecode','stad','atad','standcode','BAY','categorycode','remarkcodeflight']]
today_active_flight= selected_columns_1.copy().reset_index(drop=True)

In [14]:
#Assign all data of Bay_Stand.csv file for Bay_standcode
Bay_standcode = pd.read_csv(r'C:\Users\Lenovo\Bay_Stand.csv')

In [15]:
#Rename columns
#Remove spaces before & after Bay_standcode['BAY']
Bay_standcode = Bay_standcode.rename(columns={'bay':'BAY','terminal':'TERMINAL','oper_type':'Oper_Type'})
Bay_standcode['BAY'] = Bay_standcode['BAY'].str.strip()

In [16]:
#Merge data by left joining today_active_flight as a & Bay_standcode as b whereby 'a.BAY'=b.'BAY'
TB_Active_Flight = pd.merge(today_active_flight,Bay_standcode,how='left',on=['BAY'])
#Assign Cargo_Flight equal to TB_Active_Flight['Oper_Type'] that has value of 'Cargo/Mail'
Cargo_Flight = TB_Active_Flight[(TB_Active_Flight['Oper_Type'].isin(['Cargo/Mail']))].reset_index(drop=True)

In [17]:
#Combiane columns of TB_Active_Flight['station1']+TB_Active_Flight['station2']+TB_Active_Flight['station3']+TB_Active_Flight['station4']+TB_Active_Flight['station5']+TB_Active_Flight['station6'] into TB_Active_Flight['Station']
TB_Active_Flight['Station'] = TB_Active_Flight['station1']+TB_Active_Flight['station2']+TB_Active_Flight['station3']+TB_Active_Flight['station4']+TB_Active_Flight['station5']+TB_Active_Flight['station6']

In [18]:
print(TB_Active_Flight['Station'])

0        DEL KUL                 
1        KUL TRZ                 
2        BTU KUL                 
3        KUL PEN                 
4        KUL AOR                 
                   ...           
72184    KUL AUH                 
72185    KUL SIN                 
72186    KUL SIN                 
72187    KUL BKI                 
72188    KUL JHB                 
Name: Station, Length: 72189, dtype: object


In [19]:
#Remove space before & afyter the string value inside of TB_Active_Flight['Station']
TB_Active_Flight['Station']= TB_Active_Flight['Station'].str.strip()

In [20]:
print(TB_Active_Flight['Station'])

0        DEL KUL
1        KUL TRZ
2        BTU KUL
3        KUL PEN
4        KUL AOR
          ...   
72184    KUL AUH
72185    KUL SIN
72186    KUL SIN
72187    KUL BKI
72188    KUL JHB
Name: Station, Length: 72189, dtype: object


In [21]:
TB_Active_Flight['Station'].shape

(72189,)

In [22]:
#Assign first three characters TB_Active_Flight['Station'] to TB_Active_Flight['First_Station'] 
#Assign last three characters TB_Active_Flight['Station'] to TB_Active_Flight['First_Station'] 
TB_Active_Flight['First_Station'] = TB_Active_Flight['Station'].str.slice(stop=3)
TB_Active_Flight['Last_Station'] = TB_Active_Flight['Station'].str.slice(start=-3)

In [23]:
print(TB_Active_Flight['First_Station'])
print(TB_Active_Flight['Last_Station'])

0        DEL
1        KUL
2        BTU
3        KUL
4        KUL
        ... 
72184    KUL
72185    KUL
72186    KUL
72187    KUL
72188    KUL
Name: First_Station, Length: 72189, dtype: object
0        KUL
1        TRZ
2        KUL
3        PEN
4        AOR
        ... 
72184    AUH
72185    SIN
72186    SIN
72187    BKI
72188    JHB
Name: Last_Station, Length: 72189, dtype: object


In [24]:
#Remove the space before & after inside the value of column for TB_Active_Flight['First_Station']
#Remove the space before & after inside the value of column for TB_Active_Flight['Last_Station']
TB_Active_Flight['First_Station'] = TB_Active_Flight['First_Station'].str.strip()
TB_Active_Flight['Last_Station'] = TB_Active_Flight['Last_Station'].str.strip()

In [25]:
#Assign TB_Active_Flight['routing'] equal to TB_Active_Flight['Last_Station'] if TB_Active_Flight['First_Station'] = 'KUL' else equal to TB_Active_Flight['First_Station']
TB_Active_Flight['routing'] = TB_Active_Flight.apply(lambda row:row['Last_Station'] if row['First_Station']=='KUL' else row['First_Station'],axis=1)
print(TB_Active_Flight['routing'])

0        DEL
1        TRZ
2        BTU
3        PEN
4        AOR
        ... 
72184    AUH
72185    SIN
72186    SIN
72187    BKI
72188    JHB
Name: routing, Length: 72189, dtype: object


In [26]:
#Asssign required columns to test_df
req_cols = ['afskey','flightno','tailno','stad','leg','aircrafttypecode','routing','categorycode','TERMINAL','remarkcodeflight','iataservicetypecode']
test_df = TB_Active_Flight[req_cols]

In [27]:
print(test_df.columns)

Index(['afskey', 'flightno', 'tailno', 'stad', 'leg', 'aircrafttypecode',
       'routing', 'categorycode', 'TERMINAL', 'remarkcodeflight',
       'iataservicetypecode'],
      dtype='object')


In [28]:
#Rename columns' name
test_df.columns = ['afskey','fl_flight_no','fl_tail_registration','fl_actual_time','fl_leg','fl_aircraft_type','fl_routing','categorycode','TERMINAL','remarkcodeflight','iataservicetypecode']

In [29]:
#Convert pandas string object into pandas datetime object,
#if invalid parsing, the value will be set as NaT(Not-a-Time, indicate an unknown or missing datetime value)
test_df['fl_actual_time'] = pd.to_datetime(test_df['fl_actual_time'],errors='coerce')
print(test_df.columns)

Index(['afskey', 'fl_flight_no', 'fl_tail_registration', 'fl_actual_time',
       'fl_leg', 'fl_aircraft_type', 'fl_routing', 'categorycode', 'TERMINAL',
       'remarkcodeflight', 'iataservicetypecode'],
      dtype='object')


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8524\1035494415.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['fl_actual_time'] = pd.to_datetime(test_df['fl_actual_time'],errors='coerce')


In [30]:
#Define function to get week number in a month.
#with input parameters: dates (pd.Series): Series of dates.
#Output returns: pd.Series: Week number in a month.
#pd.to_timedelta(dates.dt.day - 1, unit='D') = Convert into timedelta in unit of days
#.dt.weekday = Extrat weekday value, eg monday=0 & sunday=6
def weekinmonth(dates):
    firstday_in_month = dates - pd.to_timedelta(dates.dt.day - 1, unit='D')
    return (dates.dt.day-1 + firstday_in_month.dt.weekday) // 7 + 1

In [31]:
test_df['fl_actual_time']

0       2021-05-28 18:45:00
1       2021-05-28 19:45:00
2       2021-05-28 14:05:00
3       2021-12-22 16:20:00
4       2021-12-22 13:20:00
                ...        
72184   2021-12-23 02:15:00
72185   2021-12-22 10:25:00
72186   2021-12-22 19:30:00
72187   2021-12-23 06:45:00
72188   2021-12-22 09:30:00
Name: fl_actual_time, Length: 72189, dtype: datetime64[ns]

In [32]:
#Extract year value of test_df['fl_actual_time'] to data['year']
#Extract month value of test_df['fl_actual_time'] to data['month']
#Extract week value of test_df['fl_actual_time'] to data['week']
#Extract day value of test_df['fl_actual_time'] to data['day']
#Extract dayofweek value of test_df['fl_actual_time'] to data['dayofweek']
#Extract hour value of test_df['fl_actual_time'] to data['hour']
test_df['year'] = test_df['fl_actual_time'].dt.year
test_df['month'] = test_df['fl_actual_time'].dt.month
test_df['week'] = test_df['fl_actual_time'].dt.week
test_df['day'] = test_df['fl_actual_time'].dt.day
test_df['dayofweek'] = test_df['fl_actual_time'].dt.dayofweek
test_df['hour'] = test_df['fl_actual_time'].dt.hour

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8524\4129883786.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['year'] = test_df['fl_actual_time'].dt.year
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8524\4129883786.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['month'] = test_df['fl_actual_time'].dt.month
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8524\4129883786.py:9: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().w

In [33]:
#Execute weekimonth function & return output function to column of test_df['WOM']
test_df['WOM'] = weekinmonth(test_df['fl_actual_time'])

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8524\2443507839.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['WOM'] = weekinmonth(test_df['fl_actual_time'])


In [34]:
test_df

,afskey,fl_flight_no,fl_tail_registration,fl_actual_time,fl_leg,fl_aircraft_type,fl_routing,categorycode,TERMINAL,remarkcodeflight,iataservicetypecode,year,month,week,day,dayofweek,hour,WOM
0,10493113.0,IX 0122,VTAXJ,2021-05-28 18:45:00,A,738,DEL,I,KLIA,CLB,C,2021,5,21,28,4,18,5
1,10493114.0,IX 0625,VTAXJ,2021-05-28 19:45:00,D,738,TRZ,I,KLIA,FDP,C,2021,5,21,28,4,19,5
2,10493149.0,MH 2743,9MMXA,2021-05-28 14:05:00,A,73H,BTU,D,KLIA,CLB,J,2021,5,21,28,4,14,5
3,10688876.0,MH 1152,9MMLK,2021-12-22 16:20:00,D,73H,PEN,D,KLIA,FDP,J,2021,12,51,22,2,16,4
4,10688885.0,MH 1208,9MMXT,2021-12-22 13:20:00,D,73H,AOR,D,KLIA,FDP,J,2021,12,51,22,2,13,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72184,10688847.0,EY 0411,A6BLX,2021-12-23 02:15:00,D,789,AUH,I,KLIA,FDP,J,2021,12,51,23,3,2,4
72185,10688854.0,SQ 0107,9VSMG,2021-12-22 10:25:00,D,359,SIN,I,KLIA,FDP,J,2021,12,51,22,2,10,4
72186,10688856.0,TR 0473,9VOJB,2021-12-22 19:30:00,D,789,SIN,I,klia2,FDP,J,2021,12,51,22,2,19,4
72187,10688857.0,AK 5108,9MAJN,2021-12-23 06:45:00,D,32A,BKI,D,klia2,FDP,J,2021,12,51,23,3,6,4


In [35]:
#Assign test_df['shift_time'] by condition to values 
#whereby if hour<8, data['shift_time']='0000 to 0800'
#else 15>hour>=8, data['shift_time']='0800 to 1500'
#else 21>hour>=15, data['shift_time']='1500 to 2100',
#else hour>=21, data['shift_time']='2100 to 2400''

conditions = [
    (test_df['hour'] < 8),
    (test_df['hour'] >= 8) & (test_df['hour'] < 15),
    (test_df['hour'] >= 15) & (test_df['hour'] < 21),
    (test_df['hour'] >= 21)
    ]

values = ['0000 to 0800', '0800 to 1500', '1500 to 2100', '2100 to 2400']

test_df['shift_time'] = np.select(conditions, values)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8524\1643356222.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['shift_time'] = np.select(conditions, values)


In [36]:
test_df

,afskey,fl_flight_no,fl_tail_registration,fl_actual_time,fl_leg,fl_aircraft_type,fl_routing,categorycode,TERMINAL,remarkcodeflight,iataservicetypecode,year,month,week,day,dayofweek,hour,WOM,shift_time
0,10493113.0,IX 0122,VTAXJ,2021-05-28 18:45:00,A,738,DEL,I,KLIA,CLB,C,2021,5,21,28,4,18,5,1500 to 2100
1,10493114.0,IX 0625,VTAXJ,2021-05-28 19:45:00,D,738,TRZ,I,KLIA,FDP,C,2021,5,21,28,4,19,5,1500 to 2100
2,10493149.0,MH 2743,9MMXA,2021-05-28 14:05:00,A,73H,BTU,D,KLIA,CLB,J,2021,5,21,28,4,14,5,0800 to 1500
3,10688876.0,MH 1152,9MMLK,2021-12-22 16:20:00,D,73H,PEN,D,KLIA,FDP,J,2021,12,51,22,2,16,4,1500 to 2100
4,10688885.0,MH 1208,9MMXT,2021-12-22 13:20:00,D,73H,AOR,D,KLIA,FDP,J,2021,12,51,22,2,13,4,0800 to 1500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72184,10688847.0,EY 0411,A6BLX,2021-12-23 02:15:00,D,789,AUH,I,KLIA,FDP,J,2021,12,51,23,3,2,4,0000 to 0800
72185,10688854.0,SQ 0107,9VSMG,2021-12-22 10:25:00,D,359,SIN,I,KLIA,FDP,J,2021,12,51,22,2,10,4,0800 to 1500
72186,10688856.0,TR 0473,9VOJB,2021-12-22 19:30:00,D,789,SIN,I,klia2,FDP,J,2021,12,51,22,2,19,4,1500 to 2100
72187,10688857.0,AK 5108,9MAJN,2021-12-23 06:45:00,D,32A,BKI,D,klia2,FDP,J,2021,12,51,23,3,6,4,0000 to 0800


In [37]:
#No csv file
agg_data = pd.read_csv('pax_agg_post_covid_IATA.csv')

In [38]:
agg_data

,fl_internal_key,fl_tail_registration,fl_leg,fl_routing,categorycode,TERMINAL,fl_type_count,mean,min,max,median,last
0,3879624.0,9MMXI,A,KCH,D,KLIA,48.0,126.125000,48.0,159.0,131.0,131.0
1,3879993.0,PKAXY,A,SUB,I,klia2,70.0,67.461538,40.0,104.0,69.0,40.0
2,3880728.0,9MLCR,D,CGK,I,KLIA,159.0,145.687500,117.0,162.0,149.5,150.0
3,3879582.0,9MLCR,A,CGK,I,KLIA,91.0,74.187500,17.0,145.0,74.0,77.0
4,3879723.0,4RANC,A,CMB,I,KLIA,109.0,111.222222,29.0,165.0,118.0,142.0
...,...,...,...,...,...,...,...,...,...,...,...,...
17109,3880408.0,9MMTO,A,HKG,I,KLIA,88.0,88.000000,88.0,88.0,88.0,88.0
17110,3880403.0,9MMXA,A,SIN,I,KLIA,39.0,39.000000,39.0,39.0,39.0,39.0
17111,3880394.0,9MMLP,A,HKT,I,KLIA,81.0,81.000000,81.0,81.0,81.0,81.0
17112,3880428.0,9MMLQ,A,PEN,D,KLIA,77.0,104.285714,62.0,139.0,104.0,82.0


In [39]:
#Merge two dataframe of test_df as a & agg_data as b by left join, 
#whereby a.fl_tail_registration = b.fl_tail_registration, a.fl_routing = b.fl_routing, a.fl_leg = b.fl_leg, a.categorycode = b.categorycode & a.TERMINAL = b.TERMINAL 
test_df = pd.merge(test_df,agg_data,how='left',on=['fl_tail_registration','fl_routing','fl_leg','categorycode','TERMINAL'])

In [40]:
#Fill nan value in column of test_df['mean'] with 0
#Fill nan value in column of test_df['min'] with 0
#Fill nan value in column of test_df['max'] with 0
#Fill nan value in column of test_df['median'] with 0
#Fill nan value in column of test_df['last'] with 0
test_df['mean'] = test_df['mean'].fillna(0)
test_df['min'] = test_df['min'].fillna(0)
test_df['max'] = test_df['max'].fillna(0)
test_df['median'] = test_df['median'].fillna(0)
test_df['last'] = test_df['last'].fillna(0)

In [41]:
#Write shape details(rows & columns) for test_df dataframe in Autamation.log
logger.info("Test Dataframe shape is {}".format(test_df.shape))

In [42]:
#Assign all data of Unique_combo_post_covid_IATA.csv file to data
unique_combo = pd.read_csv('Unique_combo_post_covid_IATA.csv')

In [43]:
pd.options.display.min_rows= 20
unique_combo

,fl_tail_registration,fl_leg,fl_routing,fl_afskey,Available
0,4RABM,A,CMB,2,Yes
1,4RABM,D,CMB,2,Yes
2,4RABN,A,CMB,8,Yes
3,4RABN,D,CMB,8,Yes
4,4RABO,A,CMB,11,Yes
5,4RABO,D,CMB,11,Yes
6,4RABQ,A,CMB,3,Yes
7,4RABQ,D,CMB,3,Yes
8,4RALA,A,CMB,1,Yes
9,4RALA,D,CMB,1,Yes


In [44]:
#Drop unmique_combo['fl_afskey'] column
unique_combo = unique_combo.drop(['fl_afskey'],axis=1)

In [45]:
unique_combo.isna().sum()

fl_tail_registration    0
fl_leg                  0
fl_routing              0
Available               0
dtype: int64

In [46]:
#Merge two dataframe of test_df as a & agg_data as b by left join, 
#whereby a.fl_tail_registration = b.fl_tail_registration, a.fl_leg = b.fl_leg, a.categorycode = b.categorycode & a.fl_routing = b.fl_routing 
test_df = pd.merge(test_df,unique_combo,how='left',on=['fl_tail_registration','fl_leg','fl_routing'])

In [47]:
test_df

,afskey,fl_flight_no,fl_tail_registration,fl_actual_time,fl_leg,fl_aircraft_type,fl_routing,categorycode,TERMINAL,remarkcodeflight,...,WOM,shift_time,fl_internal_key,fl_type_count,mean,min,max,median,last,Available
0,10493113.0,IX 0122,VTAXJ,2021-05-28 18:45:00,A,738,DEL,I,KLIA,CLB,...,5,1500 to 2100,3864820.0,56.0,56.000000,56.0,56.0,56.0,56.0,Yes
1,10493114.0,IX 0625,VTAXJ,2021-05-28 19:45:00,D,738,TRZ,I,KLIA,FDP,...,5,1500 to 2100,3865332.0,166.0,153.200000,70.0,186.0,172.0,172.0,Yes
2,10493114.0,IX 0625,VTAXJ,2021-05-28 19:45:00,D,738,TRZ,I,KLIA,FDP,...,5,1500 to 2100,3875986.0,186.0,153.200000,70.0,186.0,172.0,172.0,Yes
3,10493114.0,IX 0625,VTAXJ,2021-05-28 19:45:00,D,738,TRZ,I,KLIA,FDP,...,5,1500 to 2100,3875989.0,172.0,153.200000,70.0,186.0,172.0,172.0,Yes
4,10493114.0,IX 0625,VTAXJ,2021-05-28 19:45:00,D,738,TRZ,I,KLIA,FDP,...,5,1500 to 2100,3875990.0,172.0,153.200000,70.0,186.0,172.0,172.0,Yes
5,10493114.0,IX 0625,VTAXJ,2021-05-28 19:45:00,D,738,TRZ,I,KLIA,FDP,...,5,1500 to 2100,3877736.0,70.0,153.200000,70.0,186.0,172.0,172.0,Yes
6,10493149.0,MH 2743,9MMXA,2021-05-28 14:05:00,A,73H,BTU,D,KLIA,CLB,...,5,0800 to 1500,3880533.0,98.0,114.666667,98.0,128.0,118.0,118.0,Yes
7,10493149.0,MH 2743,9MMXA,2021-05-28 14:05:00,A,73H,BTU,D,KLIA,CLB,...,5,0800 to 1500,3880671.0,118.0,114.666667,98.0,128.0,118.0,118.0,Yes
8,10493149.0,MH 2743,9MMXA,2021-05-28 14:05:00,A,73H,BTU,D,KLIA,CLB,...,5,0800 to 1500,3875684.0,128.0,114.666667,98.0,128.0,118.0,118.0,Yes
9,10688876.0,MH 1152,9MMLK,2021-12-22 16:20:00,D,73H,PEN,D,KLIA,FDP,...,4,1500 to 2100,3882425.0,122.0,106.250000,91.0,122.0,106.0,122.0,Yes


In [48]:
#Fill nan value with 'No'
test_df['Available'] = test_df['Available'].fillna('No')

In [49]:
#Copy values of 'Yes' from test_df['Available'] into test_df_yes
#Copy values of 'No' from test_df['Available'] into test_df_no
test_df_yes = test_df[test_df['Available']=='Yes']
test_df_no = test_df[test_df['Available']=='No']

In [50]:
pd.options.display.max_rows = 26
pd.options.display.max_columns = 26
test_df.head(3)

,afskey,fl_flight_no,fl_tail_registration,fl_actual_time,fl_leg,fl_aircraft_type,fl_routing,categorycode,TERMINAL,remarkcodeflight,iataservicetypecode,year,month,...,day,dayofweek,hour,WOM,shift_time,fl_internal_key,fl_type_count,mean,min,max,median,last,Available
0,10493113.0,IX 0122,VTAXJ,2021-05-28 18:45:00,A,738,DEL,I,KLIA,CLB,C,2021,5,...,28,4,18,5,1500 to 2100,3864820.0,56.0,56.0,56.0,56.0,56.0,56.0,Yes
1,10493114.0,IX 0625,VTAXJ,2021-05-28 19:45:00,D,738,TRZ,I,KLIA,FDP,C,2021,5,...,28,4,19,5,1500 to 2100,3865332.0,166.0,153.2,70.0,186.0,172.0,172.0,Yes
2,10493114.0,IX 0625,VTAXJ,2021-05-28 19:45:00,D,738,TRZ,I,KLIA,FDP,C,2021,5,...,28,4,19,5,1500 to 2100,3875986.0,186.0,153.2,70.0,186.0,172.0,172.0,Yes


In [51]:
#Write all data of test_df into csv file of test_df_post_covid_IATA.csv whereby don't write indexs
test_df.to_csv('test_df_post_covid_IATA.csv',index=False)

In [52]:
#Write shape details(rows & columns) for test_df_yes & test_df_no dataframe in Autamation.log
logger.info("Test Dataframe shape for Yes is {} and No is {}".format(test_df_yes.shape,test_df_no.shape))

In [53]:
pd.options.display.max_rows =10
pd.options.display.max_columns =26
test_df_yes

,afskey,fl_flight_no,fl_tail_registration,fl_actual_time,fl_leg,fl_aircraft_type,fl_routing,categorycode,TERMINAL,remarkcodeflight,iataservicetypecode,year,month,...,day,dayofweek,hour,WOM,shift_time,fl_internal_key,fl_type_count,mean,min,max,median,last,Available
0,10493113.0,IX 0122,VTAXJ,2021-05-28 18:45:00,A,738,DEL,I,KLIA,CLB,C,2021,5,...,28,4,18,5,1500 to 2100,3864820.0,56.0,56.000000,56.0,56.0,56.0,56.0,Yes
1,10493114.0,IX 0625,VTAXJ,2021-05-28 19:45:00,D,738,TRZ,I,KLIA,FDP,C,2021,5,...,28,4,19,5,1500 to 2100,3865332.0,166.0,153.200000,70.0,186.0,172.0,172.0,Yes
2,10493114.0,IX 0625,VTAXJ,2021-05-28 19:45:00,D,738,TRZ,I,KLIA,FDP,C,2021,5,...,28,4,19,5,1500 to 2100,3875986.0,186.0,153.200000,70.0,186.0,172.0,172.0,Yes
3,10493114.0,IX 0625,VTAXJ,2021-05-28 19:45:00,D,738,TRZ,I,KLIA,FDP,C,2021,5,...,28,4,19,5,1500 to 2100,3875989.0,172.0,153.200000,70.0,186.0,172.0,172.0,Yes
4,10493114.0,IX 0625,VTAXJ,2021-05-28 19:45:00,D,738,TRZ,I,KLIA,FDP,C,2021,5,...,28,4,19,5,1500 to 2100,3875990.0,172.0,153.200000,70.0,186.0,172.0,172.0,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528761,10688857.0,AK 5108,9MAJN,2021-12-23 06:45:00,D,32A,BKI,D,klia2,FDP,J,2021,12,...,23,3,6,4,0000 to 0800,3870990.0,177.0,136.545455,103.0,177.0,137.0,103.0,Yes
528762,10688857.0,AK 5108,9MAJN,2021-12-23 06:45:00,D,32A,BKI,D,klia2,FDP,J,2021,12,...,23,3,6,4,0000 to 0800,3871166.0,141.0,136.545455,103.0,177.0,137.0,103.0,Yes
528763,10688867.0,MH 1051,9MMLT,2021-12-22 09:30:00,D,73H,JHB,D,KLIA,FDP,J,2021,12,...,22,2,9,4,0800 to 1500,3865832.0,48.0,70.333333,48.0,84.0,79.0,79.0,Yes
528764,10688867.0,MH 1051,9MMLT,2021-12-22 09:30:00,D,73H,JHB,D,KLIA,FDP,J,2021,12,...,22,2,9,4,0800 to 1500,3868766.0,79.0,70.333333,48.0,84.0,79.0,79.0,Yes


In [54]:
test_df_yes = test_df_yes.drop(['remarkcodeflight','iataservicetypecode','fl_flight_no','Available'], axis = 1)

In [55]:
pd.options.display.max_rows = 5
pd.options.display.max_columns = 23
test_df_yes.head(5)

,afskey,fl_tail_registration,fl_actual_time,fl_leg,fl_aircraft_type,fl_routing,categorycode,TERMINAL,year,month,week,day,dayofweek,hour,WOM,shift_time,fl_internal_key,fl_type_count,mean,min,max,median,last
0,10493113.0,VTAXJ,2021-05-28 18:45:00,A,738,DEL,I,KLIA,2021,5,21,28,4,18,5,1500 to 2100,3864820.0,56.0,56.0,56.0,56.0,56.0,56.0
1,10493114.0,VTAXJ,2021-05-28 19:45:00,D,738,TRZ,I,KLIA,2021,5,21,28,4,19,5,1500 to 2100,3865332.0,166.0,153.2,70.0,186.0,172.0,172.0
2,10493114.0,VTAXJ,2021-05-28 19:45:00,D,738,TRZ,I,KLIA,2021,5,21,28,4,19,5,1500 to 2100,3875986.0,186.0,153.2,70.0,186.0,172.0,172.0
3,10493114.0,VTAXJ,2021-05-28 19:45:00,D,738,TRZ,I,KLIA,2021,5,21,28,4,19,5,1500 to 2100,3875989.0,172.0,153.2,70.0,186.0,172.0,172.0
4,10493114.0,VTAXJ,2021-05-28 19:45:00,D,738,TRZ,I,KLIA,2021,5,21,28,4,19,5,1500 to 2100,3875990.0,172.0,153.2,70.0,186.0,172.0,172.0


In [56]:
test_df_yes = test_df_yes.rename(columns={'afskey':'fl_afskey'})

In [57]:
test_df_yes = test_df_yes[['fl_afskey','fl_internal_key','fl_tail_registration','fl_actual_time','fl_leg','fl_aircraft_type','fl_routing','categorycode','TERMINAL','fl_type_count','hour','shift_time','year','month','week','day','dayofweek','WOM','mean','min','max','median','last']]

In [58]:
pd.options.display.max_rows = 10
pd.options.display.max_columns = 23
test_df_yes.head(10)

,fl_afskey,fl_internal_key,fl_tail_registration,fl_actual_time,fl_leg,fl_aircraft_type,fl_routing,categorycode,TERMINAL,fl_type_count,hour,shift_time,year,month,week,day,dayofweek,WOM,mean,min,max,median,last
0,10493113.0,3864820.0,VTAXJ,2021-05-28 18:45:00,A,738,DEL,I,KLIA,56.0,18,1500 to 2100,2021,5,21,28,4,5,56.000000,56.0,56.0,56.0,56.0
1,10493114.0,3865332.0,VTAXJ,2021-05-28 19:45:00,D,738,TRZ,I,KLIA,166.0,19,1500 to 2100,2021,5,21,28,4,5,153.200000,70.0,186.0,172.0,172.0
2,10493114.0,3875986.0,VTAXJ,2021-05-28 19:45:00,D,738,TRZ,I,KLIA,186.0,19,1500 to 2100,2021,5,21,28,4,5,153.200000,70.0,186.0,172.0,172.0
3,10493114.0,3875989.0,VTAXJ,2021-05-28 19:45:00,D,738,TRZ,I,KLIA,172.0,19,1500 to 2100,2021,5,21,28,4,5,153.200000,70.0,186.0,172.0,172.0
4,10493114.0,3875990.0,VTAXJ,2021-05-28 19:45:00,D,738,TRZ,I,KLIA,172.0,19,1500 to 2100,2021,5,21,28,4,5,153.200000,70.0,186.0,172.0,172.0
5,10493114.0,3877736.0,VTAXJ,2021-05-28 19:45:00,D,738,TRZ,I,KLIA,70.0,19,1500 to 2100,2021,5,21,28,4,5,153.200000,70.0,186.0,172.0,172.0
6,10493149.0,3880533.0,9MMXA,2021-05-28 14:05:00,A,73H,BTU,D,KLIA,98.0,14,0800 to 1500,2021,5,21,28,4,5,114.666667,98.0,128.0,118.0,118.0
7,10493149.0,3880671.0,9MMXA,2021-05-28 14:05:00,A,73H,BTU,D,KLIA,118.0,14,0800 to 1500,2021,5,21,28,4,5,114.666667,98.0,128.0,118.0,118.0
8,10493149.0,3875684.0,9MMXA,2021-05-28 14:05:00,A,73H,BTU,D,KLIA,128.0,14,0800 to 1500,2021,5,21,28,4,5,114.666667,98.0,128.0,118.0,118.0
9,10688876.0,3882425.0,9MMLK,2021-12-22 16:20:00,D,73H,PEN,D,KLIA,122.0,16,1500 to 2100,2021,12,51,22,2,4,106.250000,91.0,122.0,106.0,122.0


In [59]:
test_df_yes.shape

(515625, 23)

In [60]:
#Import pycaret package
from pycaret.regression import *

exp_1 = setup(data=test_df_yes,target='fl_type_count',train_size=0.8,
              categorical_features=['fl_tail_registration','fl_leg','fl_routing','categorycode','TERMINAL','shift_time'],
              ignore_features=['fl_actual_time','fl_afskey','fl_internal_key','fl_aircraft_type'],
              numeric_features=['year','month','week','day','dayofweek','WOM','hour','mean','median','min','max','last'],session_id=123,silent=True
             )

C:\Users\Lenovo\.conda\envs\tpe\lib\site-packages\sktime\utils\validation\series.py:21: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  VALID_INDEX_TYPES = (pd.Int64Index, pd.RangeIndex, pd.PeriodIndex, pd.DatetimeIndex)
C:\Users\Lenovo\.conda\envs\tpe\lib\site-packages\sktime\forecasting\base\_fh.py:18: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  RELATIVE_TYPES = (pd.Int64Index, pd.RangeIndex)
C:\Users\Lenovo\.conda\envs\tpe\lib\site-packages\sktime\forecasting\base\_fh.py:19: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  ABSOLUTE_TYPES = (pd.Int64Index, pd.RangeIndex, pd.DatetimeIndex, pd.PeriodIndex)
C:\Users\Lenovo\.conda\envs\tpe\lib\site-packages\sktime\datat

,Description,Value
0,Session id,123
1,Target,fl_type_count
2,Target type,regression
3,Data shape,"(515625, 19)"
4,Train data shape,"(412500, 19)"
5,Test data shape,"(103125, 19)"
6,Ignore features,4
7,Ordinal features,2
8,Numeric features,12
9,Categorical features,6


In [61]:
#Load trained pycaret model
saved_final_lr = load_model('Final_Linear_Regression')

Transformation Pipeline and Model Successfully Loaded


In [62]:
saved_final_lr

Pipeline(memory=Memory(location=C:\Users\Lenovo\AppData\Local\Temp\joblib),
         steps=[('numerical_imputer',
                 TransfomerWrapper(include=['year', 'month', 'week', 'day', 'dayofweek', 'WOM', 'hour', 'mean', 'median', 'min', 'max', 'last'], transformer=SimpleImputer())),
                ('categorical_imputer',
                 TransfomerWrapper(include=['fl_tail_registration', 'fl_leg', 'fl_routing', 'categ...
                 TransfomerWrapper(include=['TERMINAL', 'shift_time'], transformer=OneHotEncoder(cols=['TERMINAL', 'shift_time'], handle_missing='return_nan', use_cat_names=True))),
                ('rest_encoding',
                 TransfomerWrapper(include=['fl_tail_registration', 'fl_routing'], transformer=LeaveOneOutEncoder(cols=['fl_tail_registration', 'fl_routing'], handle_missing='return_nan', random_state=123))),
                ['trained_model', LinearRegression(n_jobs=-1)]])

In [63]:
new_prediction = predict_model(saved_final_lr['trained_model'], data=test_df_yes)

In [64]:
pd.options.display.max_rows = 10
pd.options.display.max_columns = 19
new_prediction.head(10)

,fl_tail_registration,fl_leg,fl_routing,categorycode,TERMINAL,hour,shift_time,year,month,week,day,dayofweek,WOM,mean,min,max,median,last,Label
0,VTAXJ,A,DEL,I,KLIA,18,1500 to 2100,2021,5,21,28,4,5,56.000000,56.0,56.0,56.0,56.0,62.001957
1,VTAXJ,D,TRZ,I,KLIA,19,1500 to 2100,2021,5,21,28,4,5,153.200000,70.0,186.0,172.0,172.0,161.579047
2,VTAXJ,D,TRZ,I,KLIA,19,1500 to 2100,2021,5,21,28,4,5,153.200000,70.0,186.0,172.0,172.0,161.579047
3,VTAXJ,D,TRZ,I,KLIA,19,1500 to 2100,2021,5,21,28,4,5,153.200000,70.0,186.0,172.0,172.0,161.579047
4,VTAXJ,D,TRZ,I,KLIA,19,1500 to 2100,2021,5,21,28,4,5,153.200000,70.0,186.0,172.0,172.0,161.579047
5,VTAXJ,D,TRZ,I,KLIA,19,1500 to 2100,2021,5,21,28,4,5,153.200000,70.0,186.0,172.0,172.0,161.579047
6,9MMXA,A,BTU,D,KLIA,14,0800 to 1500,2021,5,21,28,4,5,114.666667,98.0,128.0,118.0,118.0,115.395037
7,9MMXA,A,BTU,D,KLIA,14,0800 to 1500,2021,5,21,28,4,5,114.666667,98.0,128.0,118.0,118.0,115.395037
8,9MMXA,A,BTU,D,KLIA,14,0800 to 1500,2021,5,21,28,4,5,114.666667,98.0,128.0,118.0,118.0,115.395037
9,9MMLK,D,PEN,D,KLIA,16,1500 to 2100,2021,12,51,22,2,4,106.250000,91.0,122.0,106.0,122.0,109.070474


In [65]:
#Assign all data of for_new_combo_post_covid_IATA.csv file to data
for_new_combo = pd.read_csv('for_new_combo_post_covid_IATA.csv')

In [66]:
for_new_combo.head(30)

,fl_leg,fl_routing,fl_type_count
0,A,ADL,175.000000
1,A,AKL,91.500000
2,A,AMS,62.357143
3,A,AOR,104.182390
4,A,ATH,1.000000
...,...,...,...
25,A,GMP,4.000000
26,A,HAN,66.125000
27,A,HDY,27.800000
28,A,HKG,86.000000


In [67]:
#Rename the column of for_new_combo['fl_type_count'] into 'Label'
for_new_combo = for_new_combo.rename(columns={'fl_type_count':'Label'})

In [68]:
for_new_combo.head(20)

,fl_leg,fl_routing,Label
0,A,ADL,175.000000
1,A,AKL,91.500000
2,A,AMS,62.357143
3,A,AOR,104.182390
4,A,ATH,1.000000
...,...,...,...
15,A,CGK,94.887597
16,A,CMB,120.560000
17,A,COK,107.200000
18,A,DAC,104.061674


In [69]:
#Merge dataframe of test_df_no & for_new_combo by left join of test_df_no as a & for_new_combo as b whereby a.fl_leg = b.fl_routing
new_prediction_mean = pd.merge(test_df_no,for_new_combo, how='left',on=['fl_leg','fl_routing'])

In [70]:
new_prediction_mean.head(20)

,afskey,fl_flight_no,fl_tail_registration,fl_actual_time,fl_leg,fl_aircraft_type,fl_routing,categorycode,TERMINAL,...,fl_internal_key,fl_type_count,mean,min,max,median,last,Available,Label
0,10688885.0,MH 1208,9MMXT,2021-12-22 13:20:00,D,73H,AOR,D,KLIA,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,No,103.578616
1,10688991.0,3K 0686,9VJSM,2021-12-22 14:25:00,D,320,SIN,I,klia2,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,No,135.371901
2,10688992.0,AK 0582,9MAHZ,2021-12-22 08:15:00,D,320,MNL,I,klia2,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,No,89.131148
3,10688994.0,AK 0824,9MRAJ,2021-12-22 12:25:00,D,32N,HKT,I,klia2,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,No,61.034483
4,10689014.0,MH 0179,9MMXC,2021-12-22 22:25:00,D,73H,CMB,I,KLIA,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,No,105.880000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,10689220.0,AK 5246,9MAGZ,2021-12-23 06:20:00,D,32A,BTU,D,klia2,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,No,110.731544
16,10689223.0,AK 5112,9MAQB,2021-12-22 08:45:00,D,320,BKI,D,klia2,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,No,135.367432
17,10689237.0,AK 5116,9MAQB,2021-12-22 18:15:00,D,320,BKI,D,klia2,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,No,135.367432
18,10689241.0,EK 0343,A6EPY,2021-12-23 01:45:00,D,77W,DXB,I,KLIA,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,No,305.487805


In [71]:
#Combine two dataframe of new_prediction & new_prediction_mean into new_prediction & allow the old index being added as new column
new_prediction = pd.concat([new_prediction,new_prediction_mean]).reset_index(drop=False)

In [72]:
#Assign nan value of new_prediction['Label'] to 0 
new_prediction['Label'] = new_prediction['Label'].fillna(0)

In [73]:
#Sorting ascending on the value of new_prediction['f1_actual_time'] & avoid the old index being added as new column
new_prediction = new_prediction.sort_values('fl_actual_time',ascending=True).reset_index(drop=True)

In [74]:
#Convert datatype of column for new_prediction['Label'] into integer
new_prediction['Label'] = new_prediction['Label'].apply(np.ceil).astype(int)

In [75]:
#Assign new_prediction['Label'] equal to o if it's value is < 0 else remain the same
new_prediction['Label'] = new_prediction.apply(lambda val: 0 if val.Label < 0 else val.Label,axis=1)

In [76]:
#Summation on all the values of column for new_prediction['Label']
predicted_Pax = new_prediction.Label.sum()

In [77]:
#Writing the date & time of today_log and value of predicted_Pax into Automation.log
logger.info("Predicted pax for {} is {}".format(today_log,predicted_Pax))
print(predicted_Pax)

67076815


In [78]:
#Assign groups of column for new_prediction['shift_time','TERMINAL','categorycode','fl_leg'] by summation of value of 'Label' & Rename column 'Label' into 'Pax'
#Assign pax_group column into ['Shift_Time','Terminal','Domestic_or_International','Leg_Type','Total_Pax_Optimal']
pax_group = new_prediction.groupby(['shift_time','TERMINAL','categorycode','fl_leg'])['Label'].sum().reset_index().rename(columns={'Label':'Pax'})
pax_group.columns = ['Shift_Time','Terminal','Domestic_or_International','Leg_Type','Total_Pax_Optimal']

In [79]:
#Read Heartbeat Analytics Dashboard Framework.xlsx into heartbeat_df
#heartbeat_df = pd.read_excel("Heartbeat Analytics Dashboard Framework.xlsx", sheet_name = 'New Data Structure')

In [80]:
#Combine dataframe of heartbeat_df & pax_group by left joining of heartbeat_df as a & pax_group as b,
#whereby a.Shift_Time = b.Shift_Time, a.Terminal = b.Terminal, a.Leg_Type = b.Leg_Type & a.Domestic_or_International = b.Domestic_or_International
#Fill nan value of HeartBeat_Pax_Data with 0
#HeartBeat_Pax_Data = pd.merge(heartbeat_df,pax_group, how='left',on=['Shift_Time','Terminal','Leg_Type','Domestic_or_International'])
#HeartBeat_Pax_Data = HeartBeat_Pax_Data.fillna(0)

In [81]:
pd.options.display.max_columns = 28
Cargo_Flight

,afskey,flightno,tailno,aircrafttypecode,leg,station1,station2,station3,station4,station5,station6,iataservicetypecode,stad,atad,standcode,BAY,categorycode,remarkcodeflight,description,description_1,pier,stand code,seq,wings,TERMINAL,serv_type,Oper_Type,note
0,10689862.0,D7 0223,9MXXU,333,A,SYD,KUL,,,,,C,2021-12-22 22:50:00.000,2021-12-22 22:39:00.000,F026,F026,I,FLD,CARGO,CARGO_KACT,F,E,???,KACT,KLIA,NaN,Cargo/Mail,NaN
1,10689863.0,D7 0669,9MXXR,333,A,BLR,KUL,,,,,C,2021-12-22 12:10:00.000,2021-12-22 12:29:00.000,F027,F027,I,FLD,CARGO,CARGO_KACT,F,F,???,KACT,KLIA,NaN,Cargo/Mail,NaN
2,10689864.0,D7 0184,9MXXQ,333,D,KUL,MAA,,,,,C,2021-12-22 09:35:00.000,2021-12-22 10:42:00.000,F027,F027,I,FDP,CARGO,CARGO_KACT,F,F,???,KACT,KLIA,NaN,Cargo/Mail,NaN
3,10689865.0,D7 0185,9MXXQ,333,A,MAA,KUL,,,,,C,2021-12-22 18:10:00.000,2021-12-22 19:22:00.000,F027,F027,I,FLD,CARGO,CARGO_KACT,F,F,???,KACT,KLIA,NaN,Cargo/Mail,NaN
4,10689866.0,D7 0670,9MXXR,333,D,KUL,BKI,,,,,C,2021-12-22 14:30:00.000,2021-12-22 14:39:00.000,F027,F027,D,FDP,CARGO,CARGO_KACT,F,F,???,KACT,KLIA,NaN,Cargo/Mail,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1936,10688495.0,D7 0692,9MXXU,333,D,KUL,HKG,,,,,C,2021-12-21 15:30:00.000,2021-12-21 15:31:00.000,F027,F027,I,FDP,CARGO,CARGO_KACT,F,F,???,KACT,KLIA,NaN,Cargo/Mail,NaN
1937,10688497.0,D7 0183,9MXXR,333,A,DEL,KUL,,,,,C,2021-12-21 22:30:00.000,2021-12-21 22:53:00.000,F026,F026,I,FLD,CARGO,CARGO_KACT,F,E,???,KACT,KLIA,NaN,Cargo/Mail,NaN
1938,10688498.0,D7 0680,9MXXQ,333,D,KUL,HKG,,,,,C,2021-12-22 00:05:00.000,2021-12-22 00:44:00.000,F027,F027,I,FDP,CARGO,CARGO_KACT,F,F,???,KACT,KLIA,NaN,Cargo/Mail,NaN
1939,10688500.0,D7 0668,9MXXR,333,D,KUL,BLR,,,,,C,2021-12-22 03:10:00.000,2021-12-22 03:23:00.000,F026,F026,I,FDP,CARGO,CARGO_KACT,F,E,???,KACT,KLIA,NaN,Cargo/Mail,NaN


In [82]:
Cargo_Flight['stad'] = pd.to_datetime(Cargo_Flight['stad'],errors='coerce')

In [83]:
#Extract hour value from Cargo_Flight['stad'] into Cargo_Flight['hour']
Cargo_Flight['hour'] = Cargo_Flight['stad'].dt.hour

In [84]:
Cargo_Flight['hour']

0       22
1       12
2        9
3       18
4       14
        ..
1936    15
1937    22
1938     0
1939     3
1940     8
Name: hour, Length: 1941, dtype: int64

In [85]:
#Assign Cargo_Flight['shift_time'] by condition to values 
#whereby if hour<8, data['shift_time']='0000 to 0800'
#else 15>hour>=8, data['shift_time']='0800 to 1500'
#else 21>hour>=15, data['shift_time']='1500 to 2100',
#else hour>=21, data['shift_time']='2100 to 2400''

conditions = [
    (Cargo_Flight['hour'] < 8),
    (Cargo_Flight['hour'] >= 8) & (Cargo_Flight['hour'] < 15),
    (Cargo_Flight['hour'] >= 15) & (Cargo_Flight['hour'] < 21),
    (Cargo_Flight['hour'] >= 21)
    ]

values = ['0000 to 0800', '0800 to 1500', '1500 to 2100', '2100 to 2400']

Cargo_Flight['shift_time'] = np.select(conditions, values)

In [86]:
#Assign group of columns Cargo_Flight['shift_time','TERMINAL','categorycode','leg'] by counting of Cargo_Flight['afskey'] & rename column 'afskey':'Cargo_Flight'
cargo_flight = Cargo_Flight.groupby(['shift_time','TERMINAL','categorycode','leg'])['afskey'].count().reset_index().rename(columns={'afskey':'Cargo_Flight'})

In [87]:
cargo_flight

,shift_time,TERMINAL,categorycode,leg,Cargo_Flight
0,0000 to 0800,KLIA,D,A,2
1,0000 to 0800,KLIA,D,D,66
2,0000 to 0800,KLIA,I,A,287
3,0000 to 0800,KLIA,I,D,266
4,0800 to 1500,KLIA,D,A,3
...,...,...,...,...,...
10,1500 to 2100,KLIA,I,A,225
11,1500 to 2100,KLIA,I,D,229
12,2100 to 2400,KLIA,D,D,1
13,2100 to 2400,KLIA,I,A,183


In [88]:
#Rename columns of cargo_flight to 'Shift_Time','Terminal','Domestic_or_International','Leg_Type','Cargo_Flight'
cargo_flight.columns = ['Shift_Time','Terminal','Domestic_or_International','Leg_Type','Cargo_Flight']

In [89]:
cargo_flight

,Shift_Time,Terminal,Domestic_or_International,Leg_Type,Cargo_Flight
0,0000 to 0800,KLIA,D,A,2
1,0000 to 0800,KLIA,D,D,66
2,0000 to 0800,KLIA,I,A,287
3,0000 to 0800,KLIA,I,D,266
4,0800 to 1500,KLIA,D,A,3
...,...,...,...,...,...
10,1500 to 2100,KLIA,I,A,225
11,1500 to 2100,KLIA,I,D,229
12,2100 to 2400,KLIA,D,D,1
13,2100 to 2400,KLIA,I,A,183


In [90]:
#Assign summation of values for cargo_flight['Cargo_Flight'] to active_cargo_flights
active_cargo_flights = cargo_flight.Cargo_Flight.sum()

In [91]:
active_cargo_flights

1941

In [92]:
#Write info into Automation.log
logger.info("Active Cargo flight for {} is {}".format(today_log,active_cargo_flights))

In [93]:
#Merge dataframe of heartbeat_df & cargo_flight by left join which heartbeat_df as a & cargo_flight as b,  
#whereby a.Shift_Time = b.Shift_Time, a.Terminal = b.Terminal, a.Leg_Type = b.Leg_Type& a.Domestic_or_International = b.Domestic_or_International
#Fill nan values of HeartBeat_Cargo_flight with 0
#HeartBeat_Cargo_flight = pd.merge(heartbeat_df,cargo_flight, how='left',on=['Shift_Time','Terminal','Leg_Type','Domestic_or_International'])
#HeartBeat_Cargo_flight = HeartBeat_Cargo_flight.fillna(0)

In [94]:
pd.options.display.max_rows = 29
new_prediction.isna().sum()

index                        0
fl_tail_registration       166
fl_leg                       0
fl_routing                   0
categorycode                 0
TERMINAL                   170
hour                         0
shift_time                   0
year                         0
month                        0
week                         0
day                          0
dayofweek                    0
WOM                          0
mean                         0
min                          0
max                          0
median                       0
last                         0
Label                        0
afskey                  515625
fl_flight_no            515625
fl_actual_time          515625
fl_aircraft_type        515625
remarkcodeflight        515727
iataservicetypecode     515625
fl_internal_key         528766
fl_type_count           528766
Available               515625
dtype: int64

In [95]:
new_prediction = new_prediction.dropna(subset=['fl_actual_time'])

In [100]:
new_prediction['fl_actual_time'][1]

Timestamp('2021-03-01 02:30:00')

In [96]:
new_prediction

,index,fl_tail_registration,fl_leg,fl_routing,categorycode,TERMINAL,hour,shift_time,year,month,week,day,dayofweek,WOM,...,min,max,median,last,Label,afskey,fl_flight_no,fl_actual_time,fl_aircraft_type,remarkcodeflight,iataservicetypecode,fl_internal_key,fl_type_count,Available
0,779,A6ENH,D,DXB,I,KLIA,1,0000 to 0800,2021,3,9,1,0,1,...,0.0,0.0,0.0,0.0,306,10433991.0,EK 0343,2021-03-01 01:45:00,77W,FDP,J,NaN,NaN,No
1,788,XYALK,D,PER,I,KLIA,2,0000 to 0800,2021,3,9,1,0,1,...,0.0,0.0,0.0,0.0,94,10434093.0,8M 1301,2021-03-01 02:30:00,319,FDP,C,NaN,NaN,No
2,778,A7BAC,D,DOH,I,KLIA,2,0000 to 0800,2021,3,9,1,0,1,...,0.0,0.0,0.0,0.0,219,10433982.0,QR 0853,2021-03-01 02:50:00,77W,FDP,J,NaN,NaN,No
3,827,9MLNH,A,DAC,I,KLIA,3,0000 to 0800,2021,3,9,1,0,1,...,0.0,0.0,0.0,0.0,105,10434331.0,OD 0161,2021-03-01 03:55:00,73J,CLB,J,NaN,NaN,No
4,789,9MAGO,D,BKI,D,klia2,7,0000 to 0800,2021,3,9,1,0,1,...,0.0,0.0,0.0,0.0,136,10434097.0,AK 5134,2021-03-01 07:15:00,32N,FDP,J,NaN,NaN,No
5,882,A7ALA,A,DOH,I,KLIA,8,0800 to 1500,2021,3,9,1,0,1,...,0.0,0.0,0.0,0.0,225,10434479.0,QR 0844,2021-03-01 08:10:00,359,CLB,J,NaN,NaN,No
6,865,9MLNH,D,KCH,D,KLIA,8,0800 to 1500,2021,3,9,1,0,1,...,0.0,0.0,0.0,0.0,124,10434419.0,OD 1606,2021-03-01 08:45:00,73J,FDP,J,NaN,NaN,No
7,902,PKAZO,A,KNO,I,klia2,9,0800 to 1500,2021,3,9,1,0,1,...,0.0,0.0,0.0,0.0,120,10434589.0,QZ 0124,2021-03-01 09:15:00,320,CLB,J,NaN,NaN,No
8,866,9MMTX,D,CAN,I,KLIA,9,0800 to 1500,2021,3,9,1,0,1,...,0.0,0.0,0.0,0.0,130,10434422.0,MH 0376,2021-03-01 09:25:00,332,FDP,J,NaN,NaN,No
9,826,PKAZG,A,SUB,I,klia2,9,0800 to 1500,2021,3,9,1,0,1,...,0.0,0.0,0.0,0.0,54,10434302.0,QZ 0320,2021-03-01 09:30:00,32A,CLB,J,NaN,NaN,No


In [157]:
#Check-In Counters

def cic_pred():
    """
    Predicts max CIC required for domestic and international pax respectively for KLIA and KLIA2 terminals.
    Done for each 8 hour window in a day. 
    Uses data from forecast of pax for that day
    """
    #!!!!!!!! FIX IN PRODUCTION !!!!!!!!!!
    #IMPORTANT ATTRIBUTES - Change after more analysis
    #% of pax arriving at 3, 2 and 1 hrs early
    PERCENTAGE_DEMAND = [20,70,10]
    #% of domestic pax using the CIC
    DOMESTIC_ACTUAL_PAX = 85
    # Avg processing time(mins) of a single pax. Calculated from extensive analysis of historical data
    PROC_TIME = 3
    
    #input data for cic prediction
    pax_forecast = new_prediction
    
    #dtype change for time column 
    pax_forecast['fl_actual_time'] = pd.to_datetime(pax_forecast['fl_actual_time'], format='%Y%m%d %H:%M:%S')
    pax_forecast.sort_values(by='fl_actual_time')    
    #Filtering for only departuring flights
    pax_forecast = pax_forecast[pax_forecast['fl_leg'] == 'D']
    
    #% of passesngers that would actually use cic
    def percentage_passengers_dom(each):
        return (each*DOMESTIC_ACTUAL_PAX)/100
    #Demand array in 24 hrs
    timer1 = [0]*24
    timer2 = [0]*24
    timer3 = [0]*24
    timer4 = [0]*24
    timertwo = 1
    
    #Iterate thgough domestic flights
    for index, row in pax_forecast.iterrows():
        #hour index based on minutes of flight departure
        if pax_forecast['fl_actual_time'][index].minute >= 30 and pax_forecast['fl_actual_time'][index].hour != 23 :
            hour_index = pax_forecast['fl_actual_time'][index].hour +1
            #print(hour_index)
        else:
            hour_index = pax_forecast['fl_actual_time'][index].hour
            #print(hour_index)
        hours_before_dep = 3
        if hour_index < 3:
            hours_before_dep = hour_index
        #Add demand to array based on % of demand of pax in a span of 3 hours
        for i in range(hours_before_dep):
            if row['TERMINAL'] == 'KLIA':
                if row['categorycode'] == 'D':
                    timer1[hour_index-i-1] += percentage_passengers_dom(pax_forecast['Label'][index])*(PERCENTAGE_DEMAND[i]/100)
                    #print('timer1 start')
                    #print(index)
                    #print(hour_index-i-1)
                    #print(tim#er1[hour_index-i-1])
                    #psd = per#centage_passengers_dom(pax_forecast['Label'][index])
                    #print(psd#)
                    #t1 = perc#entage_passengers_dom(pax_forecast['Label'][index])*(PERCENTAGE_DEMAND[i]/100)
                    #print(t1)#
                    #print('timer1 end')
                else:
                    timer2[hour_index-i-1] += pax_forecast['Label'][index]*(PERCENTAGE_DEMAND[i]/100)
                    #print(timer2[hour_index-i-1])
                    #print('timer2 ' + str(hour_index-i-1))
                    #timertwo = timertwo + 1
                    #print('timer2counter ' + str(timertwo))
            else:
                if row['categorycode'] == 'D':
                    timer3[hour_index-i-1] += percentage_passengers_dom(pax_forecast['Label'][index])*(PERCENTAGE_DEMAND[i]/100)
                    #print(timer3[hour_index-i-1])
                    #print('timer3 ' + str(hour_index))
                else:
                    timer4[hour_index-i-1] += pax_forecast['Label'][index]*(PERCENTAGE_DEMAND[i]/100)
                    #print(timer3[hour_index-i-1])
                    #print('timer3 ' + str(hour_index))
    
    #Array with predicted no of CIC for each shift 
    #SHIFTS FOR A DAY
    #0000 - 0800
    #0800 - 1500
    #1500 - 2100
    #2100 - 2400
    col = [
    '0000 to 0800',
    '0800 to 1500',
    '1500 to 2100',
    '2100 to 2400',
    ]
    CIC = pd.DataFrame(columns = ['shifts','klia_domestic','klia_international','klia2_domestic','klia2_international'])
    
    #print('timer2 data')
    #print(timer2)
    #print('0000 to 0800')
    #v1 = max(timer1[0:8])/6*(PROC_TIME/10)
    #print(v1)
    
    CIC.loc[0] = [col[0],math.ceil((max(timer1[0:8])/6)*(PROC_TIME/10)),math.ceil((max(timer2[0:8])/6)*(PROC_TIME/10)),math.ceil((max(timer3[0:8])/6)*(PROC_TIME/10)),math.ceil((max(timer4[0:8])/6)*(PROC_TIME/10))]
    CIC.loc[1] = [col[1],math.ceil((max(timer1[8:15])/6)*(PROC_TIME/10)),math.ceil((max(timer2[8:15])/6)*(PROC_TIME/10)),math.ceil((max(timer3[8:15])/6)*(PROC_TIME/10)),math.ceil((max(timer4[8:15])/6)*(PROC_TIME/10))]
    CIC.loc[2] = [col[2],math.ceil((max(timer1[15:21])/6)*(PROC_TIME/10)),math.ceil((max(timer2[15:21])/6)*(PROC_TIME/10)),math.ceil((max(timer3[15:21])/6)*(PROC_TIME/10)),math.ceil((max(timer4[15:21])/6)*(PROC_TIME/10))]
    CIC.loc[3] = [col[3],math.ceil((max(timer1[21:24])/6)*(PROC_TIME/10)),math.ceil((max(timer2[21:24])/6)*(PROC_TIME/10)),math.ceil((max(timer3[21:24])/6)*(PROC_TIME/10)),math.ceil((max(timer4[21:24])/6)*(PROC_TIME/10))]
    return CIC

In [158]:
CIC = cic_pred()

In [99]:
CIC

,shifts,klia_domestic,klia_international,klia2_domestic,klia2_international
0,0000 to 0800,1115,2349,449,367
1,0800 to 1500,600,1730,571,688
2,1500 to 2100,266,2367,453,390
3,2100 to 2400,23,1732,14,84


In [159]:
#Rename all column
CIC.columns = ['Shift_Time','KLIA_D','KLIA_I','klia2_D','klia2_I']

In [160]:
cic_summary = CIC.melt(id_vars='Shift_Time', var_name='Type',value_name='CIC_Optimal')

In [161]:
cic_summary

,Shift_Time,Type,CIC_Optimal
0,0000 to 0800,KLIA_D,1115
1,0800 to 1500,KLIA_D,600
2,1500 to 2100,KLIA_D,266
3,2100 to 2400,KLIA_D,23
4,0000 to 0800,KLIA_I,2349
5,0800 to 1500,KLIA_I,1730
6,1500 to 2100,KLIA_I,2367
7,2100 to 2400,KLIA_I,1732
8,0000 to 0800,klia2_D,449
9,0800 to 1500,klia2_D,571


In [162]:
cic_summary[['Terminal','Domestic_or_International']] = cic_summary.Type.str.split('_',expand=True)

In [163]:
cic_summary

,Shift_Time,Type,CIC_Optimal,Terminal,Domestic_or_International
0,0000 to 0800,KLIA_D,1115,KLIA,D
1,0800 to 1500,KLIA_D,600,KLIA,D
2,1500 to 2100,KLIA_D,266,KLIA,D
3,2100 to 2400,KLIA_D,23,KLIA,D
4,0000 to 0800,KLIA_I,2349,KLIA,I
5,0800 to 1500,KLIA_I,1730,KLIA,I
6,1500 to 2100,KLIA_I,2367,KLIA,I
7,2100 to 2400,KLIA_I,1732,KLIA,I
8,0000 to 0800,klia2_D,449,klia2,D
9,0800 to 1500,klia2_D,571,klia2,D


In [164]:
cic_summary = cic_summary.drop('Type',axis=1)

In [165]:
cic_summary

,Shift_Time,CIC_Optimal,Terminal,Domestic_or_International
0,0000 to 0800,1115,KLIA,D
1,0800 to 1500,600,KLIA,D
2,1500 to 2100,266,KLIA,D
3,2100 to 2400,23,KLIA,D
4,0000 to 0800,2349,KLIA,I
5,0800 to 1500,1730,KLIA,I
6,1500 to 2100,2367,KLIA,I
7,2100 to 2400,1732,KLIA,I
8,0000 to 0800,449,klia2,D
9,0800 to 1500,571,klia2,D


In [166]:
cic_summary['Leg_Type'] = 'D'

In [167]:
cic_summary

,Shift_Time,CIC_Optimal,Terminal,Domestic_or_International,Leg_Type
0,0000 to 0800,1115,KLIA,D,D
1,0800 to 1500,600,KLIA,D,D
2,1500 to 2100,266,KLIA,D,D
3,2100 to 2400,23,KLIA,D,D
4,0000 to 0800,2349,KLIA,I,D
5,0800 to 1500,1730,KLIA,I,D
6,1500 to 2100,2367,KLIA,I,D
7,2100 to 2400,1732,KLIA,I,D
8,0000 to 0800,449,klia2,D,D
9,0800 to 1500,571,klia2,D,D


In [ ]:
HeartBeat_CIC_Data = pd.merge(heartbeat_df,cic_summary, how='left',on=['Shift_Time','Terminal','Leg_Type','Domestic_or_International'])
HeartBeat_CIC_Data = HeartBeat_CIC_Data.fillna(0)

In [169]:
pax_forecast = new_prediction
pax_forecast_int = pax_forecast[pax_forecast['categorycode'] == 'I']

In [171]:
pax_forecast_int = pax_forecast_int.reset_index()

In [172]:
pax_forecast_int

,level_0,index,fl_tail_registration,fl_leg,fl_routing,categorycode,TERMINAL,hour,shift_time,year,month,week,day,dayofweek,...,min,max,median,last,Label,afskey,fl_flight_no,fl_actual_time,fl_aircraft_type,remarkcodeflight,iataservicetypecode,fl_internal_key,fl_type_count,Available
0,0,779,A6ENH,D,DXB,I,KLIA,1,0000 to 0800,2021,3,9,1,0,...,0.0,0.0,0.0,0.0,306,10433991.0,EK 0343,2021-03-01 01:45:00,77W,FDP,J,NaN,NaN,No
1,1,788,XYALK,D,PER,I,KLIA,2,0000 to 0800,2021,3,9,1,0,...,0.0,0.0,0.0,0.0,94,10434093.0,8M 1301,2021-03-01 02:30:00,319,FDP,C,NaN,NaN,No
2,2,778,A7BAC,D,DOH,I,KLIA,2,0000 to 0800,2021,3,9,1,0,...,0.0,0.0,0.0,0.0,219,10433982.0,QR 0853,2021-03-01 02:50:00,77W,FDP,J,NaN,NaN,No
3,3,827,9MLNH,A,DAC,I,KLIA,3,0000 to 0800,2021,3,9,1,0,...,0.0,0.0,0.0,0.0,105,10434331.0,OD 0161,2021-03-01 03:55:00,73J,CLB,J,NaN,NaN,No
4,5,882,A7ALA,A,DOH,I,KLIA,8,0800 to 1500,2021,3,9,1,0,...,0.0,0.0,0.0,0.0,225,10434479.0,QR 0844,2021-03-01 08:10:00,359,CLB,J,NaN,NaN,No
5,7,902,PKAZO,A,KNO,I,klia2,9,0800 to 1500,2021,3,9,1,0,...,0.0,0.0,0.0,0.0,120,10434589.0,QZ 0124,2021-03-01 09:15:00,320,CLB,J,NaN,NaN,No
6,8,866,9MMTX,D,CAN,I,KLIA,9,0800 to 1500,2021,3,9,1,0,...,0.0,0.0,0.0,0.0,130,10434422.0,MH 0376,2021-03-01 09:25:00,332,FDP,J,NaN,NaN,No
7,9,826,PKAZG,A,SUB,I,klia2,9,0800 to 1500,2021,3,9,1,0,...,0.0,0.0,0.0,0.0,54,10434302.0,QZ 0320,2021-03-01 09:30:00,32A,CLB,J,NaN,NaN,No
8,10,889,9VSWT,A,SIN,I,KLIA,9,0800 to 1500,2021,3,9,1,0,...,0.0,0.0,0.0,0.0,114,10434519.0,SQ 0106,2021-03-01 09:30:00,77W,CLB,J,NaN,NaN,No
9,11,873,PKAZO,D,KNO,I,klia2,9,0800 to 1500,2021,3,9,1,0,...,0.0,0.0,0.0,0.0,153,10434462.0,QZ 0125,2021-03-01 09:40:00,320,FDP,J,NaN,NaN,No


In [182]:
#Immigration Counters

def immigration_pred():
    """
    Predicts max immigration counters required for the 4 shifts in a day. 
    Uses data from forecast of pax for that day
    """
    #!!!!!!!! FIX IN PRODUCTION !!!!!!!!!!
    #IMPORTANT ATTRIBUTES - Change after more analysis
    #% of pax arriving at 3, 2 and 1 hrs early
    PERCENTAGE_DEMAND = [25,70,5]
    # Avg processing time(mins) of a single pax. Calculated from extensive analysis of historical data
    PROC_TIME = 1.5
    
    #input data for pax prediction
    pax_forecast = new_prediction
    
    #dtype change for time column 
    pax_forecast['fl_actual_time'] = pd.to_datetime(pax_forecast['fl_actual_time'], format='%Y%m%d %H:%M:%S')
    pax_forecast.sort_values(by='fl_actual_time')    
    #Filtering for only departuring flights
    #pax_forecast = pax_forecast[pax_forecast['fl_leg'] == 'D']
    #Filter international 
    pax_forecast_int = pax_forecast[pax_forecast['categorycode'] == 'I'] 
    pax_forecast_int = pax_forecast_int.reset_index()
    
    #Demand array in 24 hrs
    timer1 = [0]*24
    timer2 = [0]*24
    timer3 = [0]*24
    timer4 = [0]*24
    #Same as above but for international flights
    for index, row in pax_forecast_int.iterrows():
        if pax_forecast_int['fl_actual_time'][index].minute >= 30 and pax_forecast_int['fl_actual_time'][index].hour != 23 :
            hour_index = pax_forecast_int['fl_actual_time'][index].hour +1 
        else:
            hour_index = pax_forecast_int['fl_actual_time'][index].hour
        hours_before_dep = 3
        if hour_index < 3:
            hours_before_dep = hour_index
        for i in range(hours_before_dep):
            if row['TERMINAL'] == 'KLIA':
                if row['fl_leg'] == 'A':
                    timer1[hour_index-i-1] += pax_forecast_int['Label'][index]*(PERCENTAGE_DEMAND[i]/100)
                else:
                    timer2[hour_index-i-1] += pax_forecast_int['Label'][index]*(PERCENTAGE_DEMAND[i]/100)
            else:
                if row['fl_leg'] == 'A':
                    timer3[hour_index-i-1] += pax_forecast_int['Label'][index]*(PERCENTAGE_DEMAND[i]/100)
                else:
                    timer4[hour_index-i-1] += pax_forecast_int['Label'][index]*(PERCENTAGE_DEMAND[i]/100)
    #SHIFTS FOR A DAY
    #0000 - 0800
    #0800 - 1500
    #1500 - 2100
    #2100 - 2400
    col = [
    '0000 to 0800',
    '0800 to 1500',
    '1500 to 2100',
    '2100 to 2400',
    ]
    IMMIGRATION = pd.DataFrame(columns = ['shifts','klia_arrival','klia_departure','klia2_arrival','klia2_departure'])
    
    IMMIGRATION.loc[0] = [col[0],math.ceil((max(timer1[0:8])/6)*(PROC_TIME/10)),math.ceil((max(timer2[0:8])/6)*(PROC_TIME/10)),math.ceil((max(timer3[0:8])/6)*(PROC_TIME/10)),math.ceil((max(timer4[0:8])/6)*(PROC_TIME/10))]
    IMMIGRATION.loc[1] = [col[1],math.ceil((max(timer1[8:15])/6)*(PROC_TIME/10)),math.ceil((max(timer2[8:15])/6)*(PROC_TIME/10)),math.ceil((max(timer3[8:15])/6)*(PROC_TIME/10)),math.ceil((max(timer4[8:15])/6)*(PROC_TIME/10))]
    IMMIGRATION.loc[2] = [col[2],math.ceil((max(timer1[15:21])/6)*(PROC_TIME/10)),math.ceil((max(timer2[15:21])/6)*(PROC_TIME/10)),math.ceil((max(timer3[15:21])/6)*(PROC_TIME/10)),math.ceil((max(timer4[15:21])/6)*(PROC_TIME/10))]
    IMMIGRATION.loc[3] = [col[3],math.ceil((max(timer1[21:24])/6)*(PROC_TIME/10)),math.ceil((max(timer2[21:24])/6)*(PROC_TIME/10)),math.ceil((max(timer3[21:24])/6)*(PROC_TIME/10)),math.ceil((max(timer4[21:24])/6)*(PROC_TIME/10))]
    return IMMIGRATION

In [183]:
IMMIGRATION = immigration_pred()

In [184]:
IMMIGRATION

,shifts,klia_arrival,klia_departure,klia2_arrival,klia2_departure
0,0000 to 0800,747,1217,138,184
1,0800 to 1500,1199,857,125,348
2,1500 to 2100,886,1176,81,189
3,2100 to 2400,335,897,69,50


In [185]:
IMMIGRATION.columns = ['Shift_Time','KLIA_A','KLIA_D','klia2_A','klia2_D']
immi_summary = IMMIGRATION.melt(id_vars='Shift_Time', var_name='Type',value_name='Immigration_Optimal')
immi_summary[['Terminal','Leg_Type']] = immi_summary.Type.str.split('_',expand=True)
immi_summary = immi_summary.drop('Type',axis=1)
immi_summary['Domestic_or_International'] = 'I'

In [186]:
immi_summary

,Shift_Time,Immigration_Optimal,Terminal,Leg_Type,Domestic_or_International
0,0000 to 0800,747,KLIA,A,I
1,0800 to 1500,1199,KLIA,A,I
2,1500 to 2100,886,KLIA,A,I
3,2100 to 2400,335,KLIA,A,I
4,0000 to 0800,1217,KLIA,D,I
5,0800 to 1500,857,KLIA,D,I
6,1500 to 2100,1176,KLIA,D,I
7,2100 to 2400,897,KLIA,D,I
8,0000 to 0800,138,klia2,A,I
9,0800 to 1500,125,klia2,A,I


In [ ]:
HeartBeat_Immi_Data = pd.merge(heartbeat_df,immi_summary, how='left',on=['Shift_Time','Terminal','Leg_Type','Domestic_or_International'])
HeartBeat_Immi_Data = HeartBeat_Immi_Data.fillna(0)
HeartBeat_Immi_Data

In [190]:
prediction_details

,afskey,fl_flight_no,fl_actual_time,fl_tail_registration,fl_aircraft_type,fl_routing,fl_leg,categorycode,TERMINAL,remarkcodeflight,predicted_pax
0,10433991,EK 0343,2021-03-01 01:45:00,A6ENH,77W,DXB,D,I,KLIA,FDP,306
1,10434093,8M 1301,2021-03-01 02:30:00,XYALK,319,PER,D,I,KLIA,FDP,94
2,10433982,QR 0853,2021-03-01 02:50:00,A7BAC,77W,DOH,D,I,KLIA,FDP,219
3,10434331,OD 0161,2021-03-01 03:55:00,9MLNH,73J,DAC,A,I,KLIA,CLB,105
4,10434097,AK 5134,2021-03-01 07:15:00,9MAGO,32N,BKI,D,D,klia2,FDP,136
5,10434479,QR 0844,2021-03-01 08:10:00,A7ALA,359,DOH,A,I,KLIA,CLB,225
6,10434419,OD 1606,2021-03-01 08:45:00,9MLNH,73J,KCH,D,D,KLIA,FDP,124
7,10434589,QZ 0124,2021-03-01 09:15:00,PKAZO,320,KNO,A,I,klia2,CLB,120
8,10434422,MH 0376,2021-03-01 09:25:00,9MMTX,332,CAN,D,I,KLIA,FDP,130
9,10434302,QZ 0320,2021-03-01 09:30:00,PKAZG,32A,SUB,A,I,klia2,CLB,54


In [191]:
#Dataframe for detailed prediction values
selected_columns = new_prediction[['afskey','fl_flight_no','fl_actual_time','fl_tail_registration','fl_aircraft_type','fl_routing','fl_leg','categorycode','TERMINAL','remarkcodeflight','Label']]
prediction_details = selected_columns.copy().reset_index(drop=True)
prediction_details.columns = ['afskey','fl_flight_no','fl_actual_time','fl_tail_registration','fl_aircraft_type','fl_routing','fl_leg','categorycode','TERMINAL','remarkcodeflight','predicted_pax']
prediction_details['afskey'] = prediction_details['afskey'].astype(int)
prediction_details['predicted_pax'] = prediction_details['predicted_pax'].apply(np.ceil).astype(int)